In [7]:
import numpy as np
import pandas as pd
import re 
import matplotlib.pyplot as plt 
import sys
import time

#Input file must be given to run the code.
#For Example Run(in terminal) 'python3 Bioinformatics input.xlsx'

#Filename =sys.argv[1] # Read input file 

df = pd.read_excel('updatedBigDataInput.xlsx')# Read EXCEL Files and storing in database 'df'
data= pd.DataFrame()# Creating New Data Frame 
glyco_sulfide_frame= pd.DataFrame()   #Data Frame which contains Glycosilation and sulfide.

In [8]:
# Storing Usefull information From the database 'df' 
data['Entry']=df['Entry'] #Storing Entry
data['Entry name']=df['Entry name']  #Storing Entry Name
data['Protein names']=df['Protein names']  #Storing Protein Names
data['Gene names']=df['Gene names'] # Storing Gene Names
data['Length']=df['Length'] # Storing Length
data['Organism']=df['Organism'] # Storing Type of Organism
data['Mass']=df['Mass'] #Storing Mass
data['Status']=df['Status']

#Now removing the extra data we dont need and containg only organism 'Human'
human_data=data[data['Organism'].str.contains("Homo sapiens")] #Selecting only organism which contains 'Homo sapiens'
data= human_data

data['Disulfide bond']=df['Disulfide bond'] # Storing Disulfide bond 
data['Glycosylation']= df['Glycosylation']# Storing Glycosylation Position


dataframe1 = data[data['Glycosylation'].notna()].reset_index(drop=True)
dataframe2 = data[data['Disulfide bond'].notna()].reset_index(drop=True)
dataframe3 = data.dropna().reset_index(drop=True)
dataframe1=dataframe1.fillna(0)
dataframe2=dataframe2.fillna(0)

In [9]:
def get_sulfide_value(newdata):
        return re.findall('\d+\..\d+',newdata)
    
def get_int_value(data):        #function defined for global purpose use to get integer values for any column.
    return re.findall('\d+',data)

def get_float_value(data):
    return re.findall('\d+\.\d+',data)

def get_Nlinked_GlcNAc(data):
    return re.findall('\d+;  /note="N-linked \(GlcNAc...\) asparagine"',data)

def get_Nlinked_Glc(data):
    return re.findall('\d+;  /note="N-linked \(Glc\)',data)

def get_OLinked(data):
    return re.findall('\d+\;  /note=\"O-linked',data)

In [10]:
def seperateFrameBy_NAN(newdata):
    
    # Working on getting the relative positions of the Disulfide bond
    temp_disulfide= newdata['Disulfide bond'].astype(str) # Storing data temporary as 'disulfide_column'
    disulfide_column= newdata['Disulfide bond'].astype(str)
    bond= temp_disulfide.apply(get_sulfide_value) # Function call which gives all the positons 
    glyco_sulfide_frame['bond']=bond
    
    newdata['Disulfide bond']=disulfide_column.apply(get_sulfide_value)# Storing extracted disulfide value in data Frame.


    data1= newdata['Glycosylation'].astype(str) # Storing data temporary as 'Glyco_data'
    newdata['Glycosylation_GlcNac']= data1.apply(get_Nlinked_GlcNAc) # the return data with 'N-linked (GlcNAc...)'

    data2= newdata['Glycosylation'].astype(str)
    newdata['Glycosylation_Nlinked_Glc']=data2.apply(get_Nlinked_Glc)

    data3= newdata['Glycosylation'].astype(str)
    newdata['Glycosylation_Olinked']=data3.apply(get_OLinked)

    #Now Removing the extra word 'CARBOHYD' and getting all positions of Glycosylation.
    temp_data= newdata['Glycosylation_GlcNac'].astype(str)
    newdata['Glycosylation_Nlinked_GlcNac']= temp_data.apply(get_int_value) 

    temp_data= newdata['Glycosylation_Nlinked_Glc'].astype(str)
    newdata['Glycosylation_Nlinked_Glc']= temp_data.apply(get_int_value) 

    temp_data= newdata['Glycosylation_Olinked'].astype(str)
    newdata['Glycosylation_Olinked']= temp_data.apply(get_int_value)
    
    return newdata

In [11]:
dataframe1=seperateFrameBy_NAN(dataframe1)
dataframe2=seperateFrameBy_NAN(dataframe2)
dataframe3=seperateFrameBy_NAN(dataframe3)

In [127]:
dataframe1.to_excel('Glycosylation_data1.xlsx')
dataframe2.to_excel('Glycosylation_data2.xlsx')
dataframe3.to_excel('Glycosylation_data3.xlsx')

In [12]:
dataframe1

,Entry,Entry name,Protein names,Gene names,Length,Organism,Mass,Status,Disulfide bond,Glycosylation,Glycosylation_GlcNac,Glycosylation_Nlinked_Glc,Glycosylation_Olinked,Glycosylation_Nlinked_GlcNac
0,P04439,HLAA_HUMAN,"HLA class I histocompatibility antigen, A alph...",HLA-A HLAA,365,Homo sapiens (Human),"40,841",reviewed,"[125..188, 227..283]","CARBOHYD 110; /note=""N-linked (GlcNAc...) asp...","[110; /note=""N-linked (GlcNAc...) asparagine""]",[],[],[110]
1,P01911,DRB_HUMAN,"HLA class II histocompatibility antigen, DRB1 ...",HLA-DRB1,266,Homo sapiens (Human),"29,966",reviewed,"[44..108, 146..202]","CARBOHYD 48; /note=""N-linked (GlcNAc...) aspa...","[48; /note=""N-linked (GlcNAc...) asparagine""]",[],[],[48]
2,P01889,HLAB_HUMAN,"HLA class I histocompatibility antigen, B alph...",HLA-B HLAB,362,Homo sapiens (Human),"40,460",reviewed,"[125..188, 227..283]","CARBOHYD 110; /note=""N-linked (GlcNAc...) asp...","[110; /note=""N-linked (GlcNAc...) asparagine""]",[],[],[110]
3,P08246,ELNE_HUMAN,Neutrophil elastase (EC 3.4.21.37) (Bone marro...,ELANE ELA2,267,Homo sapiens (Human),"28,518",reviewed,"[55..71, 151..208, 181..187, 198..223]","CARBOHYD 88; /note=""N-linked (GlcNAc...) aspa...","[88; /note=""N-linked (GlcNAc...) asparagine"",...",[],[],"[88, 124, 173]"
4,P10144,GRAB_HUMAN,Granzyme B (EC 3.4.21.79) (C11) (CTLA-1) (Cath...,GZMB CGL1 CSPB CTLA1 GRB,247,Homo sapiens (Human),"27,716",reviewed,"[49..65, 142..209, 173..188]","CARBOHYD 71; /note=""N-linked (GlcNAc...) aspa...","[71; /note=""N-linked (GlcNAc...) asparagine"",...",[],[],"[71, 104]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4544,O00222,GRM8_HUMAN,Metabotropic glutamate receptor 8 (mGluR8),GRM8 GPRC1H MGLUR8,908,Homo sapiens (Human),"101,741",reviewed,"[64..106, 246..534, 369..384, 424..431, 516..5...","CARBOHYD 95; /note=""N-linked (GlcNAc...) aspa...","[95; /note=""N-linked (GlcNAc...) asparagine"",...",[],[],"[95, 298, 452, 480, 565]"
4545,Q8IXB1,DJC10_HUMAN,DnaJ homolog subfamily C member 10 (EC 1.8.4.-...,DNAJC10 ERDJ5 UNQ495/PRO1012,793,Homo sapiens (Human),"91,080",reviewed,"[158..161, 480..483, 588..591, 700..703]","CARBOHYD 530; /note=""N-linked (GlcNAc...) asp...","[530; /note=""N-linked (GlcNAc...) asparagine""]",[],[],[530]
4546,Q9NQ84,GPC5C_HUMAN,G-protein coupled receptor family C group 5 me...,GPRC5C RAIG3 PSEC0087,441,Homo sapiens (Human),"48,193",reviewed,[],"CARBOHYD 191; /note=""N-linked (GlcNAc...) asp...","[191; /note=""N-linked (GlcNAc...) asparagine""]",[],[],[191]
4547,Q9NZH0,GPC5B_HUMAN,G-protein coupled receptor family C group 5 me...,GPRC5B RAIG2,403,Homo sapiens (Human),"44,795",reviewed,[],"CARBOHYD 30; /note=""N-linked (GlcNAc...) aspa...","[30; /note=""N-linked (GlcNAc...) asparagine""]",[],[],[30]
